In [1]:
import numpy as np
import pandas as pd
import pickle

In [2]:
pd.set_option("display.max_columns", 101)
pd.set_option("display.max_rows", 300)

In [3]:
game_df = pd.read_pickle('data/games.pickle')

In [4]:
game_df.shape

(20508, 38)

In [5]:
game_df.columns

Index(['season_year', 'week_num', 'full_game_date', 'team', 'opp',
       'game_day_of_week', 'game_date', 'game_outcome', 'pts_off', 'pts_def',
       'pass_cmp', 'pass_att', 'pass_yds', 'pass_td', 'pass_int',
       'pass_sacked', 'pass_sacked_yds', 'pass_yds_per_att',
       'pass_net_yds_per_att', 'pass_cmp_perc', 'pass_rating', 'rush_att',
       'rush_yds', 'rush_yds_per_att', 'rush_td', 'fgm', 'fga', 'xpm', 'xpa',
       'punt', 'punt_yds', 'overtime', 'game_location', 'time_of_poss',
       'third_down_success', 'third_down_att', 'fourth_down_success',
       'fourth_down_att'],
      dtype='object')

In [6]:
mask = (game_df.team == 'chi') & (game_df.season_year == 2019)
game_df[mask]

,season_year,week_num,full_game_date,team,opp,game_day_of_week,game_date,game_outcome,pts_off,pts_def,pass_cmp,pass_att,pass_yds,pass_td,pass_int,pass_sacked,pass_sacked_yds,pass_yds_per_att,pass_net_yds_per_att,pass_cmp_perc,pass_rating,rush_att,rush_yds,rush_yds_per_att,rush_td,fgm,fga,xpm,xpa,punt,punt_yds,overtime,game_location,time_of_poss,third_down_success,third_down_att,fourth_down_success,fourth_down_att
19520,2019,1.0,2019-09-05,chi,gnb,Thu,September 5,L,3.0,10.0,26.0,45.0,208.0,0.0,1.0,5.0,20.0,5.0,4.0,57.0,60.0,15.0,46.0,3.0,0.0,1.0,1.0,0.0,0.0,8.0,341.0,NaN,NaN,28:57,3.0,15.0,0.0,2.0
19521,2019,2.0,2019-09-15,chi,den,Sun,September 15,W,16.0,14.0,16.0,27.0,120.0,0.0,0.0,0.0,0.0,4.0,4.0,59.0,70.0,29.0,153.0,5.0,1.0,3.0,3.0,1.0,1.0,5.0,287.0,NaN,@,27:01,3.0,11.0,1.0,1.0
19522,2019,3.0,2019-09-23,chi,was,Mon,September 23,W,31.0,15.0,25.0,31.0,208.0,3.0,1.0,3.0,23.0,7.0,6.0,80.0,113.0,24.0,90.0,3.0,0.0,1.0,2.0,4.0,4.0,3.0,140.0,NaN,@,31:36,8.0,13.0,0.0,0.0
19523,2019,4.0,2019-09-29,chi,min,Sun,September 29,W,16.0,6.0,24.0,33.0,197.0,1.0,0.0,1.0,7.0,6.0,5.0,72.0,97.0,33.0,72.0,2.0,0.0,3.0,3.0,1.0,1.0,5.0,241.0,NaN,NaN,35:27,5.0,16.0,2.0,3.0
19524,2019,5.0,2019-10-06,chi,rai,Sun,October 6,L,21.0,24.0,22.0,30.0,194.0,2.0,2.0,4.0,37.0,7.0,5.0,73.0,84.0,17.0,42.0,2.0,1.0,0.0,0.0,3.0,3.0,5.0,213.0,NaN,@,25:17,4.0,11.0,0.0,0.0
19525,2019,7.0,2019-10-20,chi,nor,Sun,October 20,L,25.0,36.0,34.0,54.0,235.0,2.0,0.0,2.0,16.0,4.0,4.0,63.0,85.0,7.0,17.0,2.0,0.0,1.0,1.0,2.0,2.0,7.0,263.0,NaN,NaN,22:34,2.0,12.0,1.0,2.0
19526,2019,8.0,2019-10-27,chi,sdg,Sun,October 27,L,16.0,17.0,23.0,35.0,226.0,0.0,1.0,4.0,27.0,7.0,5.0,65.0,71.0,38.0,162.0,4.0,1.0,3.0,5.0,1.0,1.0,3.0,123.0,NaN,NaN,38:00,5.0,14.0,1.0,1.0
19527,2019,9.0,2019-11-03,chi,phi,Sun,November 3,L,14.0,22.0,10.0,21.0,102.0,0.0,0.0,3.0,23.0,6.0,4.0,47.0,62.0,18.0,62.0,3.0,2.0,0.0,0.0,2.0,2.0,8.0,372.0,NaN,@,19:42,2.0,10.0,0.0,0.0
19528,2019,10.0,2019-11-10,chi,det,Sun,November 10,W,20.0,13.0,16.0,23.0,145.0,3.0,0.0,5.0,28.0,7.0,5.0,69.0,125.0,24.0,81.0,3.0,0.0,0.0,0.0,2.0,3.0,9.0,387.0,NaN,NaN,28:06,2.0,12.0,1.0,1.0
19529,2019,11.0,2019-11-17,chi,ram,Sun,November 17,L,7.0,17.0,25.0,47.0,193.0,1.0,1.0,3.0,6.0,4.0,3.0,53.0,61.0,24.0,74.0,3.0,0.0,0.0,2.0,1.0,1.0,6.0,258.0,NaN,@,32:44,6.0,17.0,0.0,2.0


In [7]:
# clean the game_location column & apply change. def home_game(row):
def home_game(row):    
    if row == '@':
        return 0
    return 1

game_df['team_home_game'] = game_df.game_location.apply(home_game)
game_df.drop("game_location", axis=1, errors='ignore', inplace=True)

In [8]:
# add team_year identifier to each row (to easily groupby team/year)

team_years = game_df['team'] + '-' +  game_df['season_year'].astype(str) 
game_df.insert(loc=2, column='team_year', value=team_years)
game_df.sample()

,season_year,week_num,team_year,full_game_date,team,opp,game_day_of_week,game_date,game_outcome,pts_off,pts_def,pass_cmp,pass_att,pass_yds,pass_td,pass_int,pass_sacked,pass_sacked_yds,pass_yds_per_att,pass_net_yds_per_att,pass_cmp_perc,pass_rating,rush_att,rush_yds,rush_yds_per_att,rush_td,fgm,fga,xpm,xpa,punt,punt_yds,overtime,time_of_poss,third_down_success,third_down_att,fourth_down_success,fourth_down_att,team_home_game
12826,2006,15.0,nor-2006,2006-12-17,nor,was,Sun,December 17,L,10.0,16.0,21.0,38.0,199.0,0.0,1.0,2.0,8.0,5.0,5.0,55.0,59.0,24.0,71.0,3.0,1.0,1.0,1.0,1.0,1.0,6.0,281.0,NaN,30:23,8.0,17.0,1.0,2.0,1


Datetime formatting strings:

- ``%B`` month fullname
- ``%-d`` day of month without leading zero\*
    - \*note: the ``-`` only works on Linux/OS-X...for windows replace with '#'
- ``%Y`` full year

In [9]:
# convert game date to datetime & add new column to DF


# f = r"%B %d-%Y"
# full_game_date = game_df.game_date + '-' + game_df.year.astype(str)
# full_game_date = pd.to_datetime(full_game_date, format=f)
# game_df.insert(loc=2, column='date', value=full_game_date)
# game_df.sample(3)

In [10]:
# now drop the game_date col
game_df.drop("game_date", axis=1, inplace=True, errors='ignore')
game_df.sample(2) # to confirm

,season_year,week_num,team_year,full_game_date,team,opp,game_day_of_week,game_outcome,pts_off,pts_def,pass_cmp,pass_att,pass_yds,pass_td,pass_int,pass_sacked,pass_sacked_yds,pass_yds_per_att,pass_net_yds_per_att,pass_cmp_perc,pass_rating,rush_att,rush_yds,rush_yds_per_att,rush_td,fgm,fga,xpm,xpa,punt,punt_yds,overtime,time_of_poss,third_down_success,third_down_att,fourth_down_success,fourth_down_att,team_home_game
3861,1988,15.0,pit-1988,1988-12-11,pit,sdg,Sun,L,14.0,20.0,17.0,33.0,172.0,1.0,2.0,3.0,34.0,6.0,4.0,51.0,51.0,24.0,130.0,5.0,1.0,0.0,0.0,2.0,2.0,5.0,221.0,NaN,26:18,NaN,NaN,NaN,NaN,0
12162,2005,2.0,htx-2005,2005-09-18,htx,pit,Sun,L,7.0,27.0,16.0,26.0,108.0,1.0,0.0,8.0,59.0,6.0,3.0,61.0,83.0,25.0,113.0,4.0,0.0,0.0,0.0,1.0,1.0,4.0,161.0,NaN,30:54,8.0,15.0,0.0,1.0,1


In [11]:
# add decade column
decades = game_df['season_year'] // 10
game_df.insert(loc=5, column='decade', value=decades)
game_df[['full_game_date', 'decade']].sample(10)

,full_game_date,decade
13798,2008-09-07,200
2362,1985-09-15,198
19921,2019-09-08,201
5048,1991-11-17,199
4724,1990-11-18,199
18597,2017-12-31,201
1752,1984-10-14,198
12663,2006-09-24,200
16032,2012-12-09,201
3844,1988-12-10,198


In [12]:
# add unique game identifier that will be the same for both teams in the same game


def apply_game_id(row):
    teams = []
    teams.append(str(row['team']))
    teams.append(str(row['opp']))
    teams.sort()
    
    game_id = teams[0] + '-' +  teams[1] + '-' + str(row['full_game_date'])[:-9]
    return game_id

In [13]:
game_ids = game_df.apply(apply_game_id, axis=1)
game_df.insert(loc=0, column='game_id', value=game_ids)

In [14]:
game_df

,game_id,season_year,week_num,team_year,full_game_date,team,decade,opp,game_day_of_week,game_outcome,pts_off,pts_def,pass_cmp,pass_att,pass_yds,pass_td,pass_int,pass_sacked,pass_sacked_yds,pass_yds_per_att,pass_net_yds_per_att,pass_cmp_perc,pass_rating,rush_att,rush_yds,rush_yds_per_att,rush_td,fgm,fga,xpm,xpa,punt,punt_yds,overtime,time_of_poss,third_down_success,third_down_att,fourth_down_success,fourth_down_att,team_home_game
0,crd-nyg-1980-09-07,1980,1.0,crd-1980,1980-09-07,crd,198,nyg,Sun,L,35.0,41.0,18.0,31.0,287.0,2.0,3.0,4.0,35.0,10.0,8.0,58.0,71.0,24.0,132.0,5.0,3.0,0.0,0.0,5.0,5.0,4.0,167.0,NaN,NaN,NaN,NaN,NaN,NaN,1
1,crd-sfo-1980-09-14,1980,2.0,crd-1980,1980-09-14,crd,198,sfo,Sun,L,21.0,24.0,28.0,38.0,256.0,2.0,1.0,2.0,9.0,7.0,6.0,73.0,98.0,37.0,86.0,2.0,1.0,0.0,1.0,3.0,3.0,7.0,253.0,OT,NaN,NaN,NaN,NaN,NaN,0
2,crd-det-1980-09-21,1980,3.0,crd-1980,1980-09-21,crd,198,det,Sun,L,7.0,20.0,22.0,36.0,242.0,1.0,1.0,5.0,33.0,7.0,5.0,61.0,78.0,18.0,82.0,4.0,0.0,0.0,1.0,1.0,1.0,6.0,275.0,NaN,NaN,NaN,NaN,NaN,NaN,0
3,crd-phi-1980-09-28,1980,4.0,crd-1980,1980-09-28,crd,198,phi,Sun,W,24.0,14.0,11.0,21.0,108.0,0.0,0.0,0.0,0.0,5.0,5.0,52.0,67.0,41.0,195.0,4.0,2.0,1.0,2.0,3.0,3.0,5.0,205.0,NaN,NaN,NaN,NaN,NaN,NaN,1
4,crd-nor-1980-10-05,1980,5.0,crd-1980,1980-10-05,crd,198,nor,Sun,W,40.0,7.0,13.0,18.0,103.0,1.0,0.0,1.0,10.0,6.0,5.0,72.0,104.0,62.0,330.0,5.0,4.0,2.0,2.0,4.0,5.0,2.0,31.0,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20503,sfo-was-2020-12-13,2020,14.0,was-2020,2020-12-13,was,202,sfo,Sun,W,23.0,15.0,15.0,32.0,95.0,0.0,1.0,2.0,13.0,3.0,2.0,46.0,40.0,28.0,98.0,3.0,0.0,3.0,4.0,2.0,2.0,8.0,398.0,NaN,27:49,3.0,15.0,0.0,0.0,0
20504,sea-was-2020-12-20,2020,15.0,was-2020,2020-12-20,was,202,sea,Sun,L,15.0,20.0,38.0,55.0,269.0,1.0,2.0,4.0,26.0,5.0,4.0,69.0,70.0,20.0,84.0,4.0,1.0,1.0,1.0,0.0,1.0,4.0,182.0,NaN,32:16,10.0,17.0,0.0,1.0,1
20505,car-was-2020-12-27,2020,16.0,was-2020,2020-12-27,was,202,car,Sun,L,13.0,20.0,26.0,47.0,278.0,1.0,2.0,3.0,13.0,6.0,5.0,55.0,62.0,19.0,108.0,5.0,0.0,2.0,2.0,1.0,1.0,3.0,154.0,NaN,24:24,7.0,15.0,1.0,3.0,1
20506,phi-was-2021-01-03,2020,17.0,was-2020,2021-01-03,was,202,phi,Sun,W,20.0,14.0,22.0,32.0,145.0,2.0,2.0,3.0,17.0,5.0,4.0,68.0,73.0,27.0,103.0,3.0,0.0,2.0,2.0,2.0,2.0,6.0,261.0,NaN,30:48,6.0,15.0,0.0,0.0,0


In [15]:
game_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20508 entries, 0 to 20507
Data columns (total 40 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   game_id               20508 non-null  object        
 1   season_year           20508 non-null  int64         
 2   week_num              20508 non-null  float64       
 3   team_year             20508 non-null  object        
 4   full_game_date        20508 non-null  datetime64[ns]
 5   team                  20508 non-null  object        
 6   decade                20508 non-null  int64         
 7   opp                   20041 non-null  object        
 8   game_day_of_week      20508 non-null  object        
 9   game_outcome          20504 non-null  object        
 10  pts_off               20504 non-null  float64       
 11  pts_def               20504 non-null  float64       
 12  pass_cmp              20504 non-null  float64       
 13  pass_att        

Alright, we need to fix a few things:

- remove rows missing 'opp' value
- convert the following cols to float:
    - pass_yds_per_att
    - pass_net_yds_per_att
    - rush_yds
    - rush_yds_per_att
    - convert overtime to boolean
    - convert time_of_poss to num
    

In [16]:
# remove missing 'opp' rows
game_df['opp'] = game_df.opp.astype(str)
game_df['opp'].replace({'nan': np.NaN}, inplace=True)
game_df.dropna(subset=['opp', 'game_outcome'], how='any', inplace=True)

In [17]:
game_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20037 entries, 0 to 20507
Data columns (total 40 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   game_id               20037 non-null  object        
 1   season_year           20037 non-null  int64         
 2   week_num              20037 non-null  float64       
 3   team_year             20037 non-null  object        
 4   full_game_date        20037 non-null  datetime64[ns]
 5   team                  20037 non-null  object        
 6   decade                20037 non-null  int64         
 7   opp                   20037 non-null  object        
 8   game_day_of_week      20037 non-null  object        
 9   game_outcome          20037 non-null  object        
 10  pts_off               20037 non-null  float64       
 11  pts_def               20037 non-null  float64       
 12  pass_cmp              20037 non-null  float64       
 13  pass_att        

In [18]:
game_df['pass_yds_per_att'].replace({str('-0'):0.0}, inplace=True)
game_df['pass_net_yds_per_att'].replace({str('-0'):0.0}, inplace=True)
game_df['pass_yds_per_att']

0        10.0
1         7.0
2         7.0
3         5.0
4         6.0
         ... 
20503     3.0
20504     5.0
20505     6.0
20506     5.0
20507     7.0
Name: pass_yds_per_att, Length: 20037, dtype: float64

In [19]:
game_df[['pass_yds', 'pass_yds_per_att', 'pass_net_yds_per_att', 'rush_yds', 'rush_yds_per_att']] = \
game_df[['pass_yds', 'pass_yds_per_att', 'pass_net_yds_per_att', 'rush_yds', 'rush_yds_per_att']].astype(float)

In [20]:
game_df.dropna(subset=['third_down_success', 'third_down_att', 'fourth_down_success', 'fourth_down_att'], how='any', inplace=True)

game_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15131 entries, 4918 to 20507
Data columns (total 40 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   game_id               15131 non-null  object        
 1   season_year           15131 non-null  int64         
 2   week_num              15131 non-null  float64       
 3   team_year             15131 non-null  object        
 4   full_game_date        15131 non-null  datetime64[ns]
 5   team                  15131 non-null  object        
 6   decade                15131 non-null  int64         
 7   opp                   15131 non-null  object        
 8   game_day_of_week      15131 non-null  object        
 9   game_outcome          15131 non-null  object        
 10  pts_off               15131 non-null  float64       
 11  pts_def               15131 non-null  float64       
 12  pass_cmp              15131 non-null  float64       
 13  pass_att     

In [21]:
# now convert overtime col to binary
game_df['overtime'].fillna(0)

4918     0
4919     0
4920     0
4921     0
4922     0
        ..
20503    0
20504    0
20505    0
20506    0
20507    0
Name: overtime, Length: 15131, dtype: object

In [22]:
game_df.overtime.value_counts()

OT    908
Name: overtime, dtype: int64

In [23]:
# convert game_outcome col to binary to avoid multicolinearity

game_df[['result_tie', 'result_win']] = pd.get_dummies(game_df.game_outcome, drop_first=True)
game_df.sample(10)

,game_id,season_year,week_num,team_year,full_game_date,team,decade,opp,game_day_of_week,game_outcome,pts_off,pts_def,pass_cmp,pass_att,pass_yds,pass_td,pass_int,pass_sacked,pass_sacked_yds,pass_yds_per_att,pass_net_yds_per_att,pass_cmp_perc,pass_rating,rush_att,rush_yds,rush_yds_per_att,rush_td,fgm,fga,xpm,xpa,punt,punt_yds,overtime,time_of_poss,third_down_success,third_down_att,fourth_down_success,fourth_down_att,team_home_game,result_tie,result_win
14980,nyg-was-2011-01-02,2010,17.0,nyg-2010,2011-01-02,nyg,201,was,Sun,W,17.0,14.0,17.0,29.0,243.0,1.0,1.0,0.0,0.0,8.0,8.0,58.0,83.0,32.0,82.0,2.0,1.0,1.0,2.0,2.0,2.0,7.0,291.0,NaN,32:20,5.0,16.0,0.0,0.0,0,0,1
18873,htx-oti-2017-10-01,2017,4.0,oti-2017,2017-10-01,oti,201,htx,Sun,L,14.0,57.0,10.0,20.0,109.0,0.0,4.0,2.0,8.0,5.0,5.0,50.0,26.0,19.0,86.0,4.0,2.0,0.0,0.0,2.0,2.0,5.0,246.0,NaN,20:19,2.0,8.0,0.0,0.0,0,0,0
11497,car-den-2004-10-10,2004,5.0,car-2004,2004-10-10,car,200,den,Sun,L,17.0,20.0,13.0,20.0,163.0,1.0,0.0,1.0,10.0,8.0,7.0,65.0,106.0,26.0,64.0,2.0,1.0,1.0,2.0,2.0,2.0,5.0,241.0,NaN,26:51,5.0,12.0,0.0,0.0,0,0,0
10344,sdg-was-2001-09-09,2001,1.0,was-2001,2001-09-09,was,200,sdg,Sun,L,3.0,30.0,15.0,32.0,117.0,0.0,2.0,1.0,2.0,3.0,3.0,46.0,30.0,19.0,44.0,2.0,0.0,1.0,1.0,0.0,0.0,7.0,287.0,NaN,23:26,2.0,12.0,0.0,1.0,0,0,0
18448,car-nor-2017-12-03,2017,13.0,car-2017,2017-12-03,car,201,nor,Sun,L,21.0,31.0,17.0,28.0,167.0,2.0,0.0,2.0,16.0,6.0,5.0,60.0,101.0,23.0,112.0,4.0,1.0,0.0,0.0,3.0,3.0,5.0,215.0,NaN,26:39,3.0,10.0,0.0,2.0,0,0,0
18185,nyg-phi-2016-12-22,2016,16.0,nyg-2016,2016-12-22,nyg,201,phi,Thu,L,19.0,24.0,38.0,63.0,356.0,1.0,3.0,0.0,0.0,5.0,5.0,60.0,61.0,25.0,114.0,4.0,0.0,4.0,4.0,1.0,1.0,3.0,145.0,NaN,34:04,10.0,22.0,1.0,2.0,0,0,0
19160,crd-kan-2018-11-11,2018,10.0,kan-2018,2018-11-11,kan,201,crd,Sun,W,26.0,14.0,21.0,28.0,212.0,2.0,0.0,5.0,37.0,8.0,6.0,75.0,119.0,23.0,118.0,5.0,1.0,2.0,2.0,2.0,2.0,5.0,233.0,NaN,28:13,4.0,11.0,0.0,0.0,1,0,1
5490,crd-dal-1992-09-20,1992,3.0,dal-1992,1992-09-20,dal,199,crd,Sun,W,31.0,20.0,14.0,21.0,263.0,3.0,0.0,0.0,0.0,12.0,12.0,66.0,149.0,38.0,150.0,3.0,1.0,1.0,2.0,4.0,4.0,3.0,152.0,NaN,34:01,7.0,13.0,1.0,1.0,1,0,1
20177,htx-min-2020-10-04,2020,4.0,htx-2020,2020-10-04,htx,202,min,Sun,L,23.0,31.0,20.0,33.0,290.0,2.0,0.0,3.0,10.0,9.0,8.0,60.0,109.0,26.0,96.0,3.0,0.0,3.0,3.0,2.0,2.0,4.0,201.0,NaN,23:29,3.0,12.0,1.0,2.0,1,0,0
10693,min-nyg-2002-11-10,2002,10.0,nyg-2002,2002-11-10,nyg,200,min,Sun,W,27.0,20.0,25.0,35.0,291.0,2.0,1.0,1.0,9.0,8.0,8.0,71.0,103.0,35.0,169.0,4.0,2.0,0.0,1.0,1.0,2.0,5.0,219.0,NaN,34:14,1.0,11.0,0.0,1.0,0,0,1


Okay, almost there. Our ties are coming through, but we get NaNs for teams that haven't tied, so we'll need to replace those with zeros.

In [24]:
# add 'margin' col
margins = game_df.pts_off - game_df.pts_def
game_df.insert(loc=11, column='margin', value=margins)
game_df[mask].head()

<ipython-input-24-c9b2cd68c5d5>:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  game_df[mask].head()


,game_id,season_year,week_num,team_year,full_game_date,team,decade,opp,game_day_of_week,game_outcome,pts_off,margin,pts_def,pass_cmp,pass_att,pass_yds,pass_td,pass_int,pass_sacked,pass_sacked_yds,pass_yds_per_att,pass_net_yds_per_att,pass_cmp_perc,pass_rating,rush_att,rush_yds,rush_yds_per_att,rush_td,fgm,fga,xpm,xpa,punt,punt_yds,overtime,time_of_poss,third_down_success,third_down_att,fourth_down_success,fourth_down_att,team_home_game,result_tie,result_win
19520,chi-gnb-2019-09-05,2019,1.0,chi-2019,2019-09-05,chi,201,gnb,Thu,L,3.0,-7.0,10.0,26.0,45.0,208.0,0.0,1.0,5.0,20.0,5.0,4.0,57.0,60.0,15.0,46.0,3.0,0.0,1.0,1.0,0.0,0.0,8.0,341.0,NaN,28:57,3.0,15.0,0.0,2.0,1,0,0
19521,chi-den-2019-09-15,2019,2.0,chi-2019,2019-09-15,chi,201,den,Sun,W,16.0,2.0,14.0,16.0,27.0,120.0,0.0,0.0,0.0,0.0,4.0,4.0,59.0,70.0,29.0,153.0,5.0,1.0,3.0,3.0,1.0,1.0,5.0,287.0,NaN,27:01,3.0,11.0,1.0,1.0,0,0,1
19522,chi-was-2019-09-23,2019,3.0,chi-2019,2019-09-23,chi,201,was,Mon,W,31.0,16.0,15.0,25.0,31.0,208.0,3.0,1.0,3.0,23.0,7.0,6.0,80.0,113.0,24.0,90.0,3.0,0.0,1.0,2.0,4.0,4.0,3.0,140.0,NaN,31:36,8.0,13.0,0.0,0.0,0,0,1
19523,chi-min-2019-09-29,2019,4.0,chi-2019,2019-09-29,chi,201,min,Sun,W,16.0,10.0,6.0,24.0,33.0,197.0,1.0,0.0,1.0,7.0,6.0,5.0,72.0,97.0,33.0,72.0,2.0,0.0,3.0,3.0,1.0,1.0,5.0,241.0,NaN,35:27,5.0,16.0,2.0,3.0,1,0,1
19524,chi-rai-2019-10-06,2019,5.0,chi-2019,2019-10-06,chi,201,rai,Sun,L,21.0,-3.0,24.0,22.0,30.0,194.0,2.0,2.0,4.0,37.0,7.0,5.0,73.0,84.0,17.0,42.0,2.0,1.0,0.0,0.0,3.0,3.0,5.0,213.0,NaN,25:17,4.0,11.0,0.0,0.0,0,0,0


In [25]:
# drop rows for bye weeks
game_df.dropna(axis=0, how='any', subset=['game_outcome'], inplace=True)


In [26]:
mask = game_df['team_year'] == 'kan-2019'
game_df[mask]

,game_id,season_year,week_num,team_year,full_game_date,team,decade,opp,game_day_of_week,game_outcome,pts_off,margin,pts_def,pass_cmp,pass_att,pass_yds,pass_td,pass_int,pass_sacked,pass_sacked_yds,pass_yds_per_att,pass_net_yds_per_att,pass_cmp_perc,pass_rating,rush_att,rush_yds,rush_yds_per_att,rush_td,fgm,fga,xpm,xpa,punt,punt_yds,overtime,time_of_poss,third_down_success,third_down_att,fourth_down_success,fourth_down_att,team_home_game,result_tie,result_win
19684,jax-kan-2019-09-08,2019,1.0,kan-2019,2019-09-08,kan,201,jax,Sun,W,40.0,14.0,26.0,25.0,34.0,378.0,3.0,0.0,0.0,0.0,11.0,11.0,73.0,139.0,26.0,113.0,4.0,1.0,4.0,4.0,4.0,4.0,1.0,51.0,NaN,31:15,5.0,10.0,0.0,0.0,0,0,1
19685,kan-rai-2019-09-15,2019,2.0,kan-2019,2019-09-15,kan,201,rai,Sun,W,28.0,18.0,10.0,30.0,44.0,436.0,4.0,0.0,2.0,7.0,10.0,9.0,68.0,130.0,22.0,31.0,1.0,0.0,0.0,0.0,4.0,4.0,5.0,196.0,NaN,32:35,8.0,14.0,0.0,0.0,0,0,1
19686,kan-rav-2019-09-22,2019,3.0,kan-2019,2019-09-22,kan,201,rav,Sun,W,33.0,5.0,28.0,27.0,37.0,363.0,3.0,0.0,1.0,11.0,10.0,9.0,73.0,130.0,25.0,140.0,5.0,1.0,2.0,2.0,3.0,4.0,2.0,88.0,NaN,27:16,5.0,9.0,0.0,1.0,1,0,1
19687,det-kan-2019-09-29,2019,4.0,kan-2019,2019-09-29,kan,201,det,Sun,W,34.0,4.0,30.0,24.0,42.0,315.0,0.0,0.0,0.0,0.0,7.0,7.0,57.0,81.0,25.0,123.0,4.0,3.0,2.0,3.0,4.0,4.0,2.0,91.0,NaN,26:05,4.0,9.0,1.0,1.0,0,0,1
19688,clt-kan-2019-10-06,2019,5.0,kan-2019,2019-10-06,kan,201,clt,Sun,L,13.0,-6.0,19.0,22.0,39.0,288.0,1.0,0.0,4.0,33.0,8.0,6.0,56.0,88.0,14.0,36.0,2.0,0.0,2.0,2.0,1.0,1.0,4.0,155.0,NaN,22:45,4.0,12.0,1.0,2.0,1,0,0
19689,htx-kan-2019-10-13,2019,6.0,kan-2019,2019-10-13,kan,201,htx,Sun,L,24.0,-7.0,31.0,19.0,35.0,256.0,3.0,1.0,1.0,17.0,7.0,7.0,54.0,94.0,11.0,53.0,4.0,0.0,1.0,2.0,3.0,3.0,2.0,119.0,NaN,20:12,4.0,8.0,0.0,0.0,1,0,0
19690,den-kan-2019-10-17,2019,7.0,kan-2019,2019-10-17,kan,201,den,Thu,W,30.0,24.0,6.0,20.0,30.0,191.0,2.0,0.0,1.0,2.0,6.0,6.0,66.0,106.0,27.0,80.0,3.0,0.0,3.0,3.0,3.0,3.0,6.0,266.0,NaN,32:13,5.0,15.0,1.0,1.0,0,0,1
19691,gnb-kan-2019-10-27,2019,8.0,kan-2019,2019-10-27,kan,201,gnb,Sun,L,24.0,-7.0,31.0,24.0,36.0,249.0,2.0,0.0,2.0,18.0,7.0,6.0,66.0,105.0,20.0,88.0,4.0,1.0,1.0,2.0,3.0,3.0,3.0,154.0,NaN,26:47,6.0,11.0,0.0,0.0,1,0,0
19692,kan-min-2019-11-03,2019,9.0,kan-2019,2019-11-03,kan,201,min,Sun,W,26.0,3.0,23.0,25.0,35.0,230.0,1.0,0.0,5.0,45.0,7.0,5.0,71.0,98.0,18.0,147.0,8.0,1.0,4.0,4.0,2.0,2.0,5.0,251.0,NaN,31:48,4.0,13.0,1.0,1.0,1,0,1
19693,kan-oti-2019-11-10,2019,10.0,kan-2019,2019-11-10,kan,201,oti,Sun,L,32.0,-3.0,35.0,36.0,51.0,433.0,3.0,0.0,2.0,13.0,8.0,8.0,70.0,115.0,25.0,97.0,3.0,0.0,4.0,5.0,2.0,3.0,1.0,26.0,NaN,37:52,7.0,14.0,1.0,2.0,0,0,0


In [27]:
# add 'prev_week' cols

cols_to_shift = ['week_num', 'result_win', 'result_tie', 'margin']

for col in cols_to_shift:
    new_col = 'prev_' + col
    game_df[new_col] = game_df.groupby('team_year')[col].apply(lambda grp: grp.shift(1))

In [28]:
# add 'off_bye' (boolean) col
    
def apply_off_bye(row):
    if row['week_num'] > 1: # playoff games have week_num = 0
        off_bye = row['week_num'] - row['prev_week_num'] == 2
    else:
        off_bye = False
    if off_bye == True:
        return 1
    return 0


game_df['off_bye'] = game_df[['week_num', 'prev_week_num']].apply(apply_off_bye, axis=1)



In [29]:
mask = game_df.team_year == 'chi-2019'
game_df[mask]

,game_id,season_year,week_num,team_year,full_game_date,team,decade,opp,game_day_of_week,game_outcome,pts_off,margin,pts_def,pass_cmp,pass_att,pass_yds,pass_td,pass_int,pass_sacked,pass_sacked_yds,pass_yds_per_att,pass_net_yds_per_att,pass_cmp_perc,pass_rating,rush_att,rush_yds,rush_yds_per_att,rush_td,fgm,fga,xpm,xpa,punt,punt_yds,overtime,time_of_poss,third_down_success,third_down_att,fourth_down_success,fourth_down_att,team_home_game,result_tie,result_win,prev_week_num,prev_result_win,prev_result_tie,prev_margin,off_bye
19520,chi-gnb-2019-09-05,2019,1.0,chi-2019,2019-09-05,chi,201,gnb,Thu,L,3.0,-7.0,10.0,26.0,45.0,208.0,0.0,1.0,5.0,20.0,5.0,4.0,57.0,60.0,15.0,46.0,3.0,0.0,1.0,1.0,0.0,0.0,8.0,341.0,NaN,28:57,3.0,15.0,0.0,2.0,1,0,0,NaN,NaN,NaN,NaN,0
19521,chi-den-2019-09-15,2019,2.0,chi-2019,2019-09-15,chi,201,den,Sun,W,16.0,2.0,14.0,16.0,27.0,120.0,0.0,0.0,0.0,0.0,4.0,4.0,59.0,70.0,29.0,153.0,5.0,1.0,3.0,3.0,1.0,1.0,5.0,287.0,NaN,27:01,3.0,11.0,1.0,1.0,0,0,1,1.0,0.0,0.0,-7.0,0
19522,chi-was-2019-09-23,2019,3.0,chi-2019,2019-09-23,chi,201,was,Mon,W,31.0,16.0,15.0,25.0,31.0,208.0,3.0,1.0,3.0,23.0,7.0,6.0,80.0,113.0,24.0,90.0,3.0,0.0,1.0,2.0,4.0,4.0,3.0,140.0,NaN,31:36,8.0,13.0,0.0,0.0,0,0,1,2.0,1.0,0.0,2.0,0
19523,chi-min-2019-09-29,2019,4.0,chi-2019,2019-09-29,chi,201,min,Sun,W,16.0,10.0,6.0,24.0,33.0,197.0,1.0,0.0,1.0,7.0,6.0,5.0,72.0,97.0,33.0,72.0,2.0,0.0,3.0,3.0,1.0,1.0,5.0,241.0,NaN,35:27,5.0,16.0,2.0,3.0,1,0,1,3.0,1.0,0.0,16.0,0
19524,chi-rai-2019-10-06,2019,5.0,chi-2019,2019-10-06,chi,201,rai,Sun,L,21.0,-3.0,24.0,22.0,30.0,194.0,2.0,2.0,4.0,37.0,7.0,5.0,73.0,84.0,17.0,42.0,2.0,1.0,0.0,0.0,3.0,3.0,5.0,213.0,NaN,25:17,4.0,11.0,0.0,0.0,0,0,0,4.0,1.0,0.0,10.0,0
19525,chi-nor-2019-10-20,2019,7.0,chi-2019,2019-10-20,chi,201,nor,Sun,L,25.0,-11.0,36.0,34.0,54.0,235.0,2.0,0.0,2.0,16.0,4.0,4.0,63.0,85.0,7.0,17.0,2.0,0.0,1.0,1.0,2.0,2.0,7.0,263.0,NaN,22:34,2.0,12.0,1.0,2.0,1,0,0,5.0,0.0,0.0,-3.0,1
19526,chi-sdg-2019-10-27,2019,8.0,chi-2019,2019-10-27,chi,201,sdg,Sun,L,16.0,-1.0,17.0,23.0,35.0,226.0,0.0,1.0,4.0,27.0,7.0,5.0,65.0,71.0,38.0,162.0,4.0,1.0,3.0,5.0,1.0,1.0,3.0,123.0,NaN,38:00,5.0,14.0,1.0,1.0,1,0,0,7.0,0.0,0.0,-11.0,0
19527,chi-phi-2019-11-03,2019,9.0,chi-2019,2019-11-03,chi,201,phi,Sun,L,14.0,-8.0,22.0,10.0,21.0,102.0,0.0,0.0,3.0,23.0,6.0,4.0,47.0,62.0,18.0,62.0,3.0,2.0,0.0,0.0,2.0,2.0,8.0,372.0,NaN,19:42,2.0,10.0,0.0,0.0,0,0,0,8.0,0.0,0.0,-1.0,0
19528,chi-det-2019-11-10,2019,10.0,chi-2019,2019-11-10,chi,201,det,Sun,W,20.0,7.0,13.0,16.0,23.0,145.0,3.0,0.0,5.0,28.0,7.0,5.0,69.0,125.0,24.0,81.0,3.0,0.0,0.0,0.0,2.0,3.0,9.0,387.0,NaN,28:06,2.0,12.0,1.0,1.0,1,0,1,9.0,0.0,0.0,-8.0,0
19529,chi-ram-2019-11-17,2019,11.0,chi-2019,2019-11-17,chi,201,ram,Sun,L,7.0,-10.0,17.0,25.0,47.0,193.0,1.0,1.0,3.0,6.0,4.0,3.0,53.0,61.0,24.0,74.0,3.0,0.0,0.0,2.0,1.0,1.0,6.0,258.0,NaN,32:44,6.0,17.0,0.0,2.0,0,0,0,10.0,1.0,0.0,7.0,0


In [30]:
game_df.columns

Index(['game_id', 'season_year', 'week_num', 'team_year', 'full_game_date',
       'team', 'decade', 'opp', 'game_day_of_week', 'game_outcome', 'pts_off',
       'margin', 'pts_def', 'pass_cmp', 'pass_att', 'pass_yds', 'pass_td',
       'pass_int', 'pass_sacked', 'pass_sacked_yds', 'pass_yds_per_att',
       'pass_net_yds_per_att', 'pass_cmp_perc', 'pass_rating', 'rush_att',
       'rush_yds', 'rush_yds_per_att', 'rush_td', 'fgm', 'fga', 'xpm', 'xpa',
       'punt', 'punt_yds', 'overtime', 'time_of_poss', 'third_down_success',
       'third_down_att', 'fourth_down_success', 'fourth_down_att',
       'team_home_game', 'result_tie', 'result_win', 'prev_week_num',
       'prev_result_win', 'prev_result_tie', 'prev_margin', 'off_bye'],
      dtype='object')

In [31]:
game_df.sort_values(['season_year', 'team', 'week_num'], inplace=True)

mask = (game_df['week_num'] == 1) & (game_df['team'] == 'chi')
game_df[mask]

,game_id,season_year,week_num,team_year,full_game_date,team,decade,opp,game_day_of_week,game_outcome,pts_off,margin,pts_def,pass_cmp,pass_att,pass_yds,pass_td,pass_int,pass_sacked,pass_sacked_yds,pass_yds_per_att,pass_net_yds_per_att,pass_cmp_perc,pass_rating,rush_att,rush_yds,rush_yds_per_att,rush_td,fgm,fga,xpm,xpa,punt,punt_yds,overtime,time_of_poss,third_down_success,third_down_att,fourth_down_success,fourth_down_att,team_home_game,result_tie,result_win,prev_week_num,prev_result_win,prev_result_tie,prev_margin,off_bye
4971,chi-min-1991-09-01,1991,1.0,chi-1991,1991-09-01,chi,199,min,Sun,W,10.0,4.0,6.0,17.0,24.0,186.0,1.0,0.0,0.0,0.0,7.0,7.0,70.0,107.0,33.0,90.0,2.0,0.0,1.0,2.0,1.0,1.0,5.0,200.0,NaN,34:58,5.0,13.0,0.0,0.0,1,0,1,NaN,NaN,NaN,NaN,0
5440,chi-det-1992-09-06,1992,1.0,chi-1992,1992-09-06,chi,199,det,Sun,W,27.0,3.0,24.0,19.0,30.0,227.0,2.0,0.0,0.0,0.0,7.0,7.0,63.0,108.0,29.0,140.0,4.0,1.0,2.0,3.0,3.0,3.0,4.0,173.0,NaN,30:51,4.0,12.0,1.0,1.0,1,0,1,NaN,NaN,NaN,NaN,0
5909,chi-nyg-1993-09-05,1993,1.0,chi-1993,1993-09-05,chi,199,nyg,Sun,L,20.0,-6.0,26.0,16.0,28.0,166.0,1.0,1.0,4.0,12.0,6.0,5.0,57.0,71.0,25.0,103.0,4.0,0.0,2.0,2.0,2.0,2.0,5.0,199.0,NaN,26:08,5.0,12.0,0.0,0.0,1,0,0,NaN,NaN,NaN,NaN,0
6376,chi-tam-1994-09-04,1994,1.0,chi-1994,1994-09-04,chi,199,tam,Sun,W,21.0,12.0,9.0,18.0,25.0,204.0,2.0,0.0,2.0,8.0,8.0,7.0,72.0,122.0,25.0,66.0,2.0,1.0,0.0,0.0,3.0,3.0,6.0,245.0,NaN,26:33,6.0,12.0,0.0,0.0,1,0,1,NaN,NaN,NaN,NaN,0
6865,chi-min-1995-09-03,1995,1.0,chi-1995,1995-09-03,chi,199,min,Sun,W,31.0,17.0,14.0,19.0,28.0,254.0,3.0,1.0,1.0,8.0,9.0,8.0,67.0,117.0,28.0,143.0,5.0,1.0,1.0,1.0,4.0,4.0,5.0,190.0,NaN,27:33,4.0,11.0,0.0,0.0,1,0,1,NaN,NaN,NaN,NaN,0
7383,chi-dal-1996-09-02,1996,1.0,chi-1996,1996-09-02,chi,199,dal,Mon,W,22.0,16.0,6.0,14.0,29.0,213.0,1.0,1.0,0.0,0.0,7.0,7.0,48.0,70.0,32.0,91.0,2.0,0.0,3.0,4.0,1.0,1.0,3.0,159.0,NaN,29:24,2.0,14.0,3.0,3.0,1,0,1,NaN,NaN,NaN,NaN,0
7882,chi-gnb-1997-09-01,1997,1.0,chi-1997,1997-09-01,chi,199,gnb,Mon,L,24.0,-14.0,38.0,17.0,41.0,172.0,1.0,2.0,3.0,20.0,4.0,3.0,41.0,41.0,30.0,164.0,5.0,2.0,1.0,1.0,1.0,1.0,6.0,288.0,NaN,32:47,5.0,16.0,1.0,2.0,0,0,0,NaN,NaN,NaN,NaN,0
8390,chi-jax-1998-09-06,1998,1.0,chi-1998,1998-09-06,chi,199,jax,Sun,L,23.0,-1.0,24.0,16.0,27.0,189.0,0.0,0.0,0.0,0.0,7.0,7.0,59.0,80.0,34.0,132.0,3.0,1.0,3.0,4.0,2.0,2.0,4.0,184.0,NaN,30:10,5.0,13.0,1.0,1.0,1,0,0,NaN,NaN,NaN,NaN,0
8887,chi-kan-1999-09-12,1999,1.0,chi-1999,1999-09-12,chi,199,kan,Sun,W,20.0,3.0,17.0,31.0,47.0,307.0,2.0,0.0,1.0,15.0,6.0,6.0,66.0,98.0,28.0,81.0,2.0,0.0,2.0,2.0,2.0,2.0,5.0,185.0,NaN,34:43,10.0,18.0,0.0,0.0,1,0,1,NaN,NaN,NaN,NaN,0
9408,chi-min-2000-09-03,2000,1.0,chi-2000,2000-09-03,chi,200,min,Sun,L,27.0,-3.0,30.0,27.0,41.0,272.0,2.0,0.0,2.0,18.0,7.0,6.0,65.0,100.0,25.0,153.0,6.0,1.0,2.0,2.0,3.0,3.0,4.0,124.0,NaN,32:34,9.0,15.0,0.0,0.0,0,0,0,NaN,NaN,NaN,NaN,0


In [32]:
# create columns containing team's season totals heading into the game
roll_cols = ['pts_off', 'margin', 'pts_def', 'pass_cmp', 'pass_att', 'pass_yds', 'pass_td',
             'pass_int', 'pass_sacked', 'pass_sacked_yds', 'pass_yds_per_att', 'pass_net_yds_per_att',
             'pass_cmp_perc', 'pass_rating', 'rush_att', 'rush_yds', 'rush_yds_per_att', 'rush_td',
             'fgm', 'fga', 'xpm', 'xpa', 'punt', 'punt_yds', 'third_down_success', 'third_down_att',
             'fourth_down_success', 'fourth_down_att', 'team_home_game', 'result_tie', 'result_win']

In [33]:
game_df[roll_cols].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15131 entries, 4934 to 20507
Data columns (total 31 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   pts_off               15131 non-null  float64
 1   margin                15131 non-null  float64
 2   pts_def               15131 non-null  float64
 3   pass_cmp              15131 non-null  float64
 4   pass_att              15131 non-null  float64
 5   pass_yds              15131 non-null  float64
 6   pass_td               15131 non-null  float64
 7   pass_int              15131 non-null  float64
 8   pass_sacked           15131 non-null  float64
 9   pass_sacked_yds       15131 non-null  float64
 10  pass_yds_per_att      15130 non-null  float64
 11  pass_net_yds_per_att  15107 non-null  float64
 12  pass_cmp_perc         15131 non-null  float64
 13  pass_rating           15131 non-null  float64
 14  rush_att              15131 non-null  float64
 15  rush_yds        

In [34]:
season_totals = ['sn_total_' + col_name for col_name in roll_cols]

game_df[season_totals] = (game_df.groupby('team_year')[roll_cols]
                      .transform(lambda x: round(x.shift(1).rolling(20, 1).sum())))


In [35]:
# get rolling wins (sum) (commented out to try using mean)

rolling_wins = (game_df.groupby('team_year')['result_win']
                      .transform(lambda x: round(x.shift(1).rolling(3).sum(), 3)))

game_df.insert(loc=53, column='roll3_num_wins', value=rolling_wins)

rolling_ties = (game_df.groupby('team_year')['result_win']
                      .transform(lambda x: round(x.shift(1).rolling(3).sum(), 3)))

game_df.insert(loc=53, column='roll3_num_ties', value=rolling_ties)

Now let's add more columns with **Expontentially Weighted Moving Averages (EWMA)** of our stats. This is similar to taking a rolling mean of a team's previous 6 performances, but EWMA weights the most recent game in the window much higher than the week before that, and so on.

For this, we will use a 3-16 week span (greedy). My hypothesis is that the most recent performances are the best predictors, and this should help further quantify recent performance.

In [36]:
# add ewma cols

ewma3_cols = ['ewma3_' + col_name for col_name in roll_cols]

game_df[ewma3_cols] = (game_df.groupby('team_year')[roll_cols]
                      .transform(lambda x: round(x.shift(1).ewm(span=3, min_periods=1).mean(), 3)))

ewma19_cols = ['ewma19_' + col_name for col_name in roll_cols]

game_df[ewma19_cols] = (game_df.groupby('team_year')[roll_cols]
                      .transform(lambda x: round(x.shift(1).ewm(span=19, min_periods=3).mean(), 3)))

In [37]:
np.mean([197, 120, 208])

175.0

Now, let's delete the first three rows of each team-year. We have to do this because we will be using at least some rolling columns for our model.

In [38]:
game_df.dropna(axis=0, how="any", subset=["ewma19_pass_yds"], inplace=True)

Looks good!!

**Now, we have to solve a pretty complex problem.**

Currently, each row has all the stats needed for the team in the 'team' column. However, we don't have the same information for the opponent in the same row. Let's make that happen.

To further clarify, for each row, we need to pull the opposing team's rolling stats, adding them back into the same row. This way, our model will only need one row to make a prediction.

Let's take a look at an example.

In [39]:
mask = (game_df.game_id == 'chi-rai-2019-10-06')
game_df[mask]


,game_id,season_year,week_num,team_year,full_game_date,team,decade,opp,game_day_of_week,game_outcome,pts_off,margin,pts_def,pass_cmp,pass_att,pass_yds,pass_td,pass_int,pass_sacked,pass_sacked_yds,pass_yds_per_att,pass_net_yds_per_att,pass_cmp_perc,pass_rating,rush_att,rush_yds,rush_yds_per_att,rush_td,fgm,fga,xpm,xpa,punt,punt_yds,overtime,time_of_poss,third_down_success,third_down_att,fourth_down_success,fourth_down_att,team_home_game,result_tie,result_win,prev_week_num,prev_result_win,prev_result_tie,prev_margin,off_bye,sn_total_pts_off,sn_total_margin,...,ewma3_pass_cmp_perc,ewma3_pass_rating,ewma3_rush_att,ewma3_rush_yds,ewma3_rush_yds_per_att,ewma3_rush_td,ewma3_fgm,ewma3_fga,ewma3_xpm,ewma3_xpa,ewma3_punt,ewma3_punt_yds,ewma3_third_down_success,ewma3_third_down_att,ewma3_fourth_down_success,ewma3_fourth_down_att,ewma3_team_home_game,ewma3_result_tie,ewma3_result_win,ewma19_pts_off,ewma19_margin,ewma19_pts_def,ewma19_pass_cmp,ewma19_pass_att,ewma19_pass_yds,ewma19_pass_td,ewma19_pass_int,ewma19_pass_sacked,ewma19_pass_sacked_yds,ewma19_pass_yds_per_att,ewma19_pass_net_yds_per_att,ewma19_pass_cmp_perc,ewma19_pass_rating,ewma19_rush_att,ewma19_rush_yds,ewma19_rush_yds_per_att,ewma19_rush_td,ewma19_fgm,ewma19_fga,ewma19_xpm,ewma19_xpa,ewma19_punt,ewma19_punt_yds,ewma19_third_down_success,ewma19_third_down_att,ewma19_fourth_down_success,ewma19_fourth_down_att,ewma19_team_home_game,ewma19_result_tie,ewma19_result_win
19524,chi-rai-2019-10-06,2019,5.0,chi-2019,2019-10-06,chi,201,rai,Sun,L,21.0,-3.0,24.0,22.0,30.0,194.0,2.0,2.0,4.0,37.0,7.0,5.0,73.0,84.0,17.0,42.0,2.0,1.0,0.0,0.0,3.0,3.0,5.0,213.0,NaN,25:17,4.0,11.0,0.0,0.0,0,0,0,4.0,1.0,0.0,10.0,0,66.0,21.0,...,71.4,95.200,28.867,85.867,2.733,0.133,2.333,2.6,1.733,1.733,4.667,226.867,5.400,14.467,1.2,1.867,0.6,0.0,0.933,17.170,6.082,11.088,22.801,33.607,184.074,1.076,0.474,2.136,12.294,5.579,4.814,67.852,86.985,25.887,90.277,3.18,0.236,2.053,2.314,1.573,1.573,5.113,246.600,4.890,13.825,0.817,1.532,0.503,0.0,0.788
19807,chi-rai-2019-10-06,2019,5.0,rai-2019,2019-10-06,rai,201,chi,Sun,W,24.0,3.0,21.0,25.0,32.0,229.0,0.0,0.0,0.0,0.0,7.0,7.0,78.0,96.0,39.0,169.0,4.0,3.0,1.0,1.0,3.0,3.0,4.0,170.0,NaN,34:43,6.0,12.0,1.0,1.0,1,0,1,4.0,1.0,0.0,7.0,0,79.0,-23.0,...,70.4,98.467,26.800,147.467,4.733,0.667,0.733,1.0,3.000,3.000,4.733,236.933,4.933,12.133,1.2,1.467,0.2,0.0,0.600,20.121,-5.742,25.863,23.253,32.374,207.790,1.552,0.733,2.044,12.038,6.874,5.824,72.095,96.013,24.950,128.855,4.55,0.715,0.738,1.000,2.558,2.558,4.550,220.944,5.772,12.633,1.050,1.311,0.448,0.0,0.503


So, in the above table, we need to get all of the roll3_x and ewma_x columns from the adjacent tables, respectively. We'll prefix these with opp_.

In [40]:
[col for col in game_df.columns]

['game_id',
 'season_year',
 'week_num',
 'team_year',
 'full_game_date',
 'team',
 'decade',
 'opp',
 'game_day_of_week',
 'game_outcome',
 'pts_off',
 'margin',
 'pts_def',
 'pass_cmp',
 'pass_att',
 'pass_yds',
 'pass_td',
 'pass_int',
 'pass_sacked',
 'pass_sacked_yds',
 'pass_yds_per_att',
 'pass_net_yds_per_att',
 'pass_cmp_perc',
 'pass_rating',
 'rush_att',
 'rush_yds',
 'rush_yds_per_att',
 'rush_td',
 'fgm',
 'fga',
 'xpm',
 'xpa',
 'punt',
 'punt_yds',
 'overtime',
 'time_of_poss',
 'third_down_success',
 'third_down_att',
 'fourth_down_success',
 'fourth_down_att',
 'team_home_game',
 'result_tie',
 'result_win',
 'prev_week_num',
 'prev_result_win',
 'prev_result_tie',
 'prev_margin',
 'off_bye',
 'sn_total_pts_off',
 'sn_total_margin',
 'sn_total_pts_def',
 'sn_total_pass_cmp',
 'sn_total_pass_att',
 'roll3_num_ties',
 'roll3_num_wins',
 'sn_total_pass_yds',
 'sn_total_pass_td',
 'sn_total_pass_int',
 'sn_total_pass_sacked',
 'sn_total_pass_sacked_yds',
 'sn_total_pass_yd

In [41]:
ERROR ME OUT

SyntaxError: invalid syntax (<ipython-input-41-4f143bb0d88c>, line 1)

In [ ]:
# gather one list of columns to get from the other game record.

opp_pull_cols = [col for col in roll_cols]

prefixes = ['prev_', 'sn_to', 'roll3', 'ewma3', 'ewma1']

[opp_pull_cols.append(col) for col in game_df.columns if col[:5] in prefixes]
opp_pull_cols.insert(0, "opp")
opp_pull_cols.insert(0, "team")
opp_pull_cols.insert(0, "game_id")
opp_pull_cols = list(set(opp_pull_cols))
opp_pull_cols

In [ ]:
game_df.opp

In [ ]:
# # convert all cols to float

# game_df[opp_pull_cols] = game_df[opp_pull_cols].fillna(0)
# for col in opp_pull_cols[3:]:
#     game_df[col] = game_df[col].astype(float)

**Let's solve this using `pd.merge`, using a self-join.**

In [ ]:
game_df = (game_df
             .merge(right=game_df[opp_pull_cols],
                    left_on=['game_id', 'team'],
                    right_on=['game_id', 'opp'],
                    suffixes=[None, '_opp']))

Let's make sure it worked by checking a couple of random game_ids.

In [ ]:
mask = (game_df.game_id == 'dal-nyg-2019-11-04')

game_df[mask][['game_id', 'team', 'opp', 'season_year', 'result_win', 'result_win_opp', 
               'pts_off', 'sn_total_pts_off', 'ewma19_pts_off', 'pts_off_opp',
               'sn_total_pts_off_opp', 'ewma19_pts_off_opp']]
              

In [ ]:
mask = (game_df.game_id == 'chi-rai-2019-10-06')

game_df[mask][['game_id', 'team', 'opp', 'season_year', 'result_win', 'result_win_opp', 
               'pts_off', 'sn_total_pts_off', 'ewma19_pts_off', 'pts_off_opp',
               'sn_total_pts_off_opp', 'ewma19_pts_off_opp']]

## Now let's take another look to make sure everything looks good.

If it does, we're ready to start modeling!

In [ ]:
mask = game_df['team_year'] == 'chi-2019'
game_df[mask]

Looks great. Now it's time to start modeling our data! Head over to `model_exploration.ipynb` to continue following along.